In [1]:
import pandas as pd

df = pd.read_csv("../data/raw_data.csv")

df.head()


,Order_ID,Customer_ID,Customer_Age,Customer_Gender,City,Area,Restaurant_ID,Restaurant_Name,Cuisine_Type,Order_Date,...,Final_Amount,Payment_Mode,Order_Status,Cancellation_Reason,Delivery_Partner_ID,Delivery_Rating,Restaurant_Rating,Order_Day,Peak_Hour,Profit_Margin
0,ORD000001,CUST6948,19.0,Male,NaN,Central,RES936,Restaurant_29,Chinese,10/20/2024,...,NaN,UPI,Delivered,NaN,DP563,5.0,4.4,Weekend,True,0.13
1,ORD000002,CUST6515,NaN,Female,Chennai,North,RES689,Restaurant_419,Chinese,8/12/2024,...,4849.0,COD,Delivered,NaN,DP369,5.0,4.7,Weekday,True,0.48
2,ORD000003,CUST1765,NaN,Male,Delhi,NaN,RES723,Restaurant_244,Arabian,12/8/2024,...,737.0,Wallet,Delivered,NaN,DP580,4.0,4.9,Weekend,True,0.08
3,ORD000004,CUST2744,NaN,Male,Mumbai,Central,RES951,Restaurant_178,Chinese,10/8/2024,...,NaN,UPI,Cancelled,Late Delivery,DP155,2.0,3.4,Weekday,NaN,0.04
4,ORD000005,CUST4389,57.0,Female,Chennai,South,RES419,Restaurant_262,Chinese,2/4/2024,...,352.0,Card,Delivered,NaN,DP728,2.0,4.4,Weekend,False,0.12


In [2]:
# Dataset size
df.shape

# Column info
df.info()

# Missing values
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Order_ID             100000 non-null  object 
 1   Customer_ID          100000 non-null  object 
 2   Customer_Age         49907 non-null   float64
 3   Customer_Gender      75144 non-null   object 
 4   City                 83274 non-null   object 
 5   Area                 83315 non-null   object 
 6   Restaurant_ID        100000 non-null  object 
 7   Restaurant_Name      100000 non-null  object 
 8   Cuisine_Type         83115 non-null   object 
 9   Order_Date           98986 non-null   object 
 10  Order_Time           98002 non-null   object 
 11  Delivery_Time_Min    66641 non-null   float64
 12  Distance_km          66530 non-null   float64
 13  Order_Value          66673 non-null   float64
 14  Discount_Applied     83285 non-null   float64
 15  Final_Amount      

Order_ID                   0
Customer_ID                0
Customer_Age           50093
Customer_Gender        24856
City                   16726
Area                   16685
Restaurant_ID              0
Restaurant_Name            0
Cuisine_Type           16885
Order_Date              1014
Order_Time              1998
Delivery_Time_Min      33359
Distance_km            33470
Order_Value            33327
Discount_Applied       16715
Final_Amount           55697
Payment_Mode           19911
Order_Status               0
Cancellation_Reason    90969
Delivery_Partner_ID        0
Delivery_Rating        16523
Restaurant_Rating          0
Order_Day                  0
Peak_Hour              32962
Profit_Margin              0
dtype: int64

In [3]:
# Fill missing customer age with median
df['Customer_Age'] = df['Customer_Age'].fillna(df['Customer_Age'].median())

# Fill missing delivery time with average
df['Delivery_Time_Min'] = df['Delivery_Time_Min'].fillna(df['Delivery_Time_Min'].mean())

# Fill missing restaurant rating with average
df['Restaurant_Rating'] = df['Restaurant_Rating'].fillna(df['Restaurant_Rating'].mean())

# Fill cancellation reason
df['Cancellation_Reason'] = df['Cancellation_Reason'].fillna("Not Cancelled")


In [4]:
# Ratings should not be more than 5
df = df[df['Delivery_Rating'] <= 5]
df = df[df['Restaurant_Rating'] <= 5]

# Order values should be positive
df = df[df['Order_Value'] > 0]
df = df[df['Final_Amount'] > 0]

# Profit should not be negative
df = df[df['Profit_Margin'] >= 0]


In [5]:
# Cap very high delivery times
max_time = df['Delivery_Time_Min'].quantile(0.95)
df.loc[df['Delivery_Time_Min'] > max_time, 'Delivery_Time_Min'] = max_time


In [6]:
# Convert order date
df['Order_Date'] = pd.to_datetime(df['Order_Date'])

# Weekday / Weekend
df['Order_Day'] = df['Order_Date'].dt.dayofweek
df['Order_Day'] = df['Order_Day'].apply(lambda x: 'Weekend' if x >= 5 else 'Weekday')


In [7]:
# Peak hour check
df['Order_Time'] = pd.to_datetime(df['Order_Time']).dt.time

df['Peak_Hour'] = df['Order_Time'].astype(str).between("18:00:00", "22:00:00")


In [8]:
# Profit margin percentage
df['Profit_Margin_Percent'] = (df['Profit_Margin'] / df['Final_Amount']) * 100


In [9]:
# Customer age group
df['Customer_Age_Group'] = pd.cut(
    df['Customer_Age'],
    bins=[18, 25, 35, 45, 60, 100],
    labels=['18-25', '26-35', '36-45', '46-60', '60+']
)


In [10]:
df.head()
df.isnull().sum()


Order_ID                    0
Customer_ID                 0
Customer_Age                0
Customer_Gender          5416
City                     3613
Area                     3576
Restaurant_ID               0
Restaurant_Name             0
Cuisine_Type             3650
Order_Date                208
Order_Time                407
Delivery_Time_Min           0
Distance_km              7147
Order_Value                 0
Discount_Applied            0
Final_Amount                0
Payment_Mode             4253
Order_Status                0
Cancellation_Reason         0
Delivery_Partner_ID         0
Delivery_Rating             0
Restaurant_Rating           0
Order_Day                   0
Peak_Hour                   0
Profit_Margin               0
Profit_Margin_Percent       0
Customer_Age_Group        235
dtype: int64

In [11]:
df.to_csv("../data/cleaned_data.csv", index=False)

print("Cleaned data saved successfully")


Cleaned data saved successfully


In [12]:
# Total orders
len(df)

# Orders by city
df['City'].value_counts().head()

# Orders by cuisine
df['Cuisine_Type'].value_counts().head()

# Cancellation percentage
(df['Order_Status'].value_counts(normalize=True) * 100)


Order_Status
Delivered    84.746316
Cancelled    15.253684
Name: proportion, dtype: float64